In [ ]:
import requests, pandas as pd, json
from bs4 import BeautifulSoup

page=requests.get("https://www.google.com/finance/markets/most-active", verify=False)
soup=BeautifulSoup(page.content,'html.parser')
stocks_table=[]
trending_table=soup.find('ul',class_="sbnBtf")

for row in trending_table.find_all('li'):
  stock_data=[]
  for cell_class in ["COaKTb","ZvmM7","BAftM","JwB6zf"]:
    if cell_class=="JwB6zf":
      if stock_data[2].split("$")[0]=="-":
        stock_data.append("-"+row.find('div',class_=cell_class).text.strip())
      else:
        stock_data.append("+"+row.find('div',class_=cell_class).text.strip())
    else:
      stock_data.append(row.find('div',class_=cell_class).text.strip())
  stocks_table.append(stock_data)

df1=pd.DataFrame(stocks_table, columns=["TICKER","COMPANY NAME","CHANGE(IN USD)","CHANGE(IN %)"])
display(df1)


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,TICKER,COMPANY NAME,CHANGE(IN USD),CHANGE(IN %)
0,2344,Winbond Electronics Corp,+$1.00,+6.43%
1,2408,Nanya Technology Corp,+$2.85,+8.30%
2,2317,Hon Hai Precision Industry Co Ltd,+$6.00,+4.08%
3,2891,CTBC Financial Holding Co Ltd,+$0.15,+0.38%
4,3481,Innolux Corp,+$0.15,+1.08%
5,2610,China Airlines Ltd,-$0.35,-1.55%
6,1605,Walsin Lihwa Corp,+$0.65,+3.04%
7,2888,Shin Kong Financial Holding Co Ltd,+$0.050,+0.44%
8,6757,Tigerair Taiwan Co Ltd,-$8.30,-7.69%
9,2882,Cathay Financial Holding Co Ltd,+$0.90,+1.63%


In [ ]:
#Part 2
base_url = 'https://api.twelvedata.com'
api_key="d4df6215c4074ad2b4e20c1b4da458ee"
endpoint = '/time_series?'
interval = '1day'
outputsize = 1
api_data=[]

#substitute with for ticker in stocks_df['Tickers]:
for ticker in df1['TICKER']:
  query = 'apikey={}&interval={}&symbol={}&outputsize={}'.format(api_key,
                                                              interval,
                                                              ticker,
                                                              outputsize)
  for value in requests.get(base_url + endpoint + query).json()['values']:

    for k, v in value.items():
      if k=="volume":
        value[k]="{:,}".format(int(value[k]))
      elif k!="datetime":
        value[k]="${:.2f}".format(float(value[k]))
    value['Ticker']=ticker
    api_data.append(value)

#Create df2 and rename columns
df2=pd.DataFrame(api_data)
df2.rename(columns={
    'datetime': 'DATE',
    'open': 'OPEN',
    'high': 'HIGH',
    'low': 'LOW',
    'close': 'CLOSE',
    'volume': 'VOLUME',
    'Ticker': 'TICKER'
}, inplace=True)
df2


,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,TICKER
0,2025-04-29,$14.12,$14.50,$13.68,$13.76,"3,878,001",SOFI
1,2025-04-29,$37.69,$37.69,$33.73,$33.82,"1,843,570",HIMS
2,2025-04-29,$8.95,$8.96,$8.94,$8.94,"1,090,985",DNB
3,2025-04-29,$107.67,$109.71,$107.44,$108.74,"1,777,562",NVDA
4,2025-04-29,$285.46,$289.27,$279.48,$280.64,"1,199,668",TSLA
5,2025-04-29,$22.83,$23.52,$22.47,$23.21,"1,400,546",PFE
6,2025-04-29,$113.98,$115.81,$113.24,$114.30,"1,135,125",PLTR
7,2025-04-29,$10.06,$10.12,$9.93,$10.03,"1,774,378",F
8,2025-04-29,$11.80,$12.13,$11.62,$11.83,"3,017,127",SOXL
9,2025-04-29,$2.52,$2.56,$2.46,$2.48,"638,969",LCID


In [ ]:
# Create df3 and reorder
df3 = pd.merge(df1, df2, on='TICKER', how='left', sort=False)
df3 = df3[['DATE','TICKER', 'COMPANY NAME','OPEN','LOW',\
            'HIGH','CLOSE','CHANGE(IN USD)','CHANGE(IN %)','VOLUME']]
df3

,DATE,TICKER,COMPANY NAME,OPEN,LOW,HIGH,CLOSE,CHANGE(IN USD),CHANGE(IN %),VOLUME
0,2025-04-29,SOFI,SoFi Technologies Inc,$14.12,$13.68,$14.50,$13.76,+$0.78,+5.91%,"3,878,001"
1,2025-04-29,HIMS,Hims & Hers Health Inc,$37.69,$33.73,$37.69,$33.82,+$6.96,+24.44%,"1,843,570"
2,2025-04-29,DNB,Dun & Bradstreet Holdings Inc,$8.95,$8.94,$8.96,$8.94,-$0.20,-2.18%,"1,090,985"
3,2025-04-29,NVDA,NVIDIA Corp,$107.67,$107.44,$109.71,$108.74,+$0.50,+0.46%,"1,777,562"
4,2025-04-29,TSLA,Tesla Inc,$285.46,$279.48,$289.27,$280.64,+$1.31,+0.46%,"1,199,668"
5,2025-04-29,PFE,Pfizer Inc,$22.83,$22.47,$23.52,$23.21,+$0.41,+1.76%,"1,400,546"
6,2025-04-29,PLTR,Palantir Technologies Inc,$113.98,$113.24,$115.81,$114.30,+$0.48,+0.42%,"1,135,125"
7,2025-04-29,F,Ford Motor Co,$10.06,$9.93,$10.12,$10.03,+$0.025,+0.25%,"1,774,378"
8,2025-04-29,SOXL,Direxion Daily Semiconductor Bull 3X Shares,$11.80,$11.62,$12.13,$11.83,-$0.31,-2.52%,"3,017,127"
9,2025-04-29,LCID,Lucid Group Inc,$2.52,$2.46,$2.56,$2.48,-$0.018,-0.72%,"638,969"


In [ ]:
# Using Describe
def clean_numeric_columns(df):
    cleaned_df = df.copy()
    for col in cleaned_df.columns:
        try:
            cleaned_df[col] = cleaned_df[col].str.replace('$', '', regex=False)\
                                             .str.replace('%', '', regex=False)\
                                             .str.replace(',', '', regex=False)\
                                             .astype(float)\
                                             .round(2)
        except (AttributeError, ValueError):
            pass
    return cleaned_df

summary_df=clean_numeric_columns(df3).describe().round(2)
summary_df

,OPEN,LOW,HIGH,CLOSE,CHANGE(IN USD),CHANGE(IN %),VOLUME
count,50.00,50.00,50.00,50.00,50.00,50.00,50.00
mean,67.68,66.72,68.50,67.46,-0.84,0.24,695966.98
std,109.66,108.01,111.18,109.49,7.26,4.08,736054.98
min,2.52,2.46,2.56,2.48,-50.45,-8.44,50927.00
25%,10.13,9.71,10.17,9.77,-0.16,-0.86,262142.00
50%,27.62,27.46,27.92,27.88,0.00,-0.02,434900.00
75%,66.03,64.33,66.87,64.72,0.30,0.61,892876.25
max,555.02,540.11,565.00,551.68,6.96,24.44,3878001.00


In [ ]:
# exporting merged dataframe to csv
df3.to_csv("df3.csv",index=False)